In [1]:
import pandas as pd
import numpy as np
import requests
import urllib.request
import json
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.wkt import loads

In [2]:
key = '01e50810998b0acda3ea2cba960a804ac68205ea'

In [3]:
baseUrl = "https://api.census.gov/data/2021/acs/acs5"
API_key = f'&key={key}'

In [4]:
#B19025 : Aggregate household income in the past 12 months (in 2022 inflation-adjusted dollars) --> i_0

# B25089: Aggregate Selected Monthly Owner Costs (Dollars) By Mortgage Status

# B19001 Household Income In The Past 12 Months (In 2021 Inflation-Adjusted Dollars) —> # households
# B25003: Tenure
# B25120: Aggregate Household Income In The Past 12 Months (In 2021 Inflation-Adjusted Dollars) By Tenure And Mortgage Status


In [5]:
# define URL for the Data Sets endpoint 
variables_bg = "?get=B19025_001E,B25003_001E,B25003_002E"
geo_bg = "&for=block%20group:*&in=county:019&in=state:45"

# open the URL as defined above and create a the request object 
request_1 = urllib.request.urlopen(baseUrl + variables_bg + geo_bg + API_key)

# actually read the data
result_variables_bg = request_1.read()

In [6]:
# define URL for the Data Sets endpoint 
variables_tract = "?get=B25120_002E,B25003_002E"
geo_tract = "&for=tract:*&in=county:019&in=state:45"

# open the URL as defined above and create a the request object 
request_2 = urllib.request.urlopen(baseUrl + variables_tract + geo_tract + API_key)

# actually read the data
result_variables_tract = request_2.read()

In [7]:
# transform to Python dictionary
jsonData_bg = json.loads(result_variables_bg.decode('utf-8'))
df_bg = pd.DataFrame(jsonData_bg)

In [8]:
# transform to Python dictionary
jsonData_tract = json.loads(result_variables_tract.decode('utf-8'))
df_tract = pd.DataFrame(jsonData_tract)

In [9]:
# first row are the column names, this needs to be fixed
df_tract.columns = df_tract.iloc[0]
df_tract = df_tract.iloc[1:]

In [10]:
# first row are the column names, this needs to be fixed
df_bg.columns = df_bg.iloc[0]
df_bg = df_bg.iloc[1:]

In [11]:
df_tract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 1 to 99
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   B25120_002E  97 non-null     object
 1   B25003_002E  99 non-null     object
 2   state        99 non-null     object
 3   county       99 non-null     object
 4   tract        99 non-null     object
dtypes: object(5)
memory usage: 4.0+ KB


In [12]:
df_tract = df_tract.rename(columns={'B25003_002E': 'owner_households_tract', 'B25120_002E': 'total_owner_i_0_tract'})

In [13]:
df_tract.head()

,total_owner_i_0_tract,owner_households_tract,state,county,tract
1,160095300,664,45,019,000100
2,165931300,506,45,019,000200
3,112160100,313,45,019,000400
4,86889200,471,45,019,000500
5,30888400,136,45,019,000600


In [14]:
merged_df = pd.merge(df_bg, df_tract[['state', 'county', 'tract', 'total_owner_i_0_tract', 'owner_households_tract']], on=['state', 'county', 'tract'], how='left')
merged_df

,B19025_001E,B25003_001E,B25003_002E,state,county,tract,block group,total_owner_i_0_tract,owner_households_tract
0,42430300,303,165,45,019,000100,1,160095300,664
1,63660700,316,254,45,019,000100,2,160095300,664
2,68497300,262,245,45,019,000100,3,160095300,664
3,55171400,243,195,45,019,000200,1,165931300,506
4,126362700,366,311,45,019,000200,2,165931300,506
...,...,...,...,...,...,...,...,...,...
256,201103800,1786,1088,45,019,005800,3,262790600,2096
257,72364500,519,369,45,019,005900,1,166856500,1139
258,39654400,406,237,45,019,005900,2,166856500,1139
259,98738400,844,533,45,019,005900,3,166856500,1139


In [15]:
merged_df['GEOID'] = merged_df["state"] + merged_df["county"] + merged_df["tract"] + merged_df['block group']

In [16]:
# Remove columns
df = merged_df.drop(columns = ["state", "county", "tract", "block group"]) 

In [17]:
df = df.apply(pd.to_numeric, errors='coerce')

In [18]:
df['fraction'] = df['B25003_002E']/df['B25003_001E']
df['owner_i_0_tract'] = df['total_owner_i_0_tract']/df['owner_households_tract']

In [19]:
df.head(2)

,B19025_001E,B25003_001E,B25003_002E,total_owner_i_0_tract,owner_households_tract,GEOID,fraction,owner_i_0_tract
0,42430300.0,303,165,160095300.0,664,450190001001,0.544554,241107.379518
1,63660700.0,316,254,160095300.0,664,450190001002,0.803797,241107.379518


In [20]:
# rename columns
df = df.rename(columns={'B19025_001E': 'i_0','B25003_001E':'#_households',
                         'B25003_002E':'owner_households'})
df.head(2)

,i_0,#_households,owner_households,total_owner_i_0_tract,owner_households_tract,GEOID,fraction,owner_i_0_tract
0,42430300.0,303,165,160095300.0,664,450190001001,0.544554,241107.379518
1,63660700.0,316,254,160095300.0,664,450190001002,0.803797,241107.379518


In [21]:
columns_to_drop = ['owner_households_tract', 'total_owner_i_0_tract']
df = df.drop(columns=columns_to_drop)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 261 entries, 0 to 260
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   i_0               260 non-null    float64
 1   #_households      261 non-null    int64  
 2   owner_households  261 non-null    int64  
 3   GEOID             261 non-null    int64  
 4   fraction          260 non-null    float64
 5   owner_i_0_tract   256 non-null    float64
dtypes: float64(3), int64(3)
memory usage: 14.3 KB


## load in shapefiles

In [23]:
df_damage_clean = pd.read_csv('data/damage_shape.csv')

#### Merge with shapefile

In [24]:
# merge with shapefile
merged_df = df.merge(df_damage_clean, on = "GEOID")

In [25]:
merged_df['geometry'] = merged_df['geometry'].apply(loads)
merged_df = gpd.GeoDataFrame(merged_df, geometry="geometry", crs='EPSG:4326')

### Check on NaN values and other issues

In [26]:
merged_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 252 entries, 0 to 251
Data columns (total 10 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   i_0                              251 non-null    float64 
 1   #_households                     252 non-null    int64   
 2   owner_households                 252 non-null    int64   
 3   GEOID                            252 non-null    int64   
 4   fraction                         252 non-null    float64 
 5   owner_i_0_tract                  247 non-null    float64 
 6   geometry                         252 non-null    geometry
 7   Max Potential Damage: Structure  252 non-null    float64 
 8   Damage: Structure                252 non-null    float64 
 9   Risk (EAD)                       252 non-null    float64 
dtypes: float64(6), geometry(1), int64(3)
memory usage: 21.7 KB


There is 1 NaN value in the i_0 \n
There needs to be checked if there block groups where there are only renter-occupied households

In [27]:
# check for block groups with no owner-occupied households
merged_df[merged_df['fraction'] <= 0]

,i_0,#_households,owner_households,GEOID,fraction,owner_i_0_tract,geometry,Max Potential Damage: Structure,Damage: Structure,Risk (EAD)
80,26100400.0,447,0,450190026122,0.0,1.365660e+05,"POLYGON ((-80.01533 32.81004, -80.01526 32.810...",7.402060e+07,0.00,104882.38
125,10836500.0,283,0,450190031112,0.0,6.162297e+04,"POLYGON ((-80.05482 32.87007, -80.05456 32.870...",3.113501e+07,0.00,603.35
133,30435500.0,704,0,450190031161,0.0,1.004359e+05,"POLYGON ((-80.04747 32.95361, -80.04690 32.953...",6.178572e+07,0.00,0.00
138,12449600.0,204,0,450190032001,0.0,-inf,"POLYGON ((-80.07008 32.89062, -80.06995 32.890...",1.396080e+07,0.00,0.00
139,14323800.0,192,0,450190032002,0.0,-inf,"POLYGON ((-80.07504 32.89815, -80.07071 32.900...",2.526424e+07,0.00,0.00
230,4864400.0,289,0,450190053003,0.0,1.760267e+05,"POLYGON ((-79.94320 32.80281, -79.94224 32.803...",2.892206e+07,1825987.65,376714.93


In [28]:
# drop the block groups with no owner-occupied households
owners_df = merged_df.loc[merged_df['fraction'] > 0]

In [29]:
# calculate average household income
owners_df['ave_i_0'] = owners_df['i_0']/owners_df['#_households']

/Users/kariendejonge/opt/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [30]:
# check on nan values
owners_df[owners_df['ave_i_0'].isna()]

,i_0,#_households,owner_households,GEOID,fraction,owner_i_0_tract,geometry,Max Potential Damage: Structure,Damage: Structure,Risk (EAD),ave_i_0
46,NaN,166,87,450190020093,0.524096,101163.326785,"POLYGON ((-79.97351 32.67128, -79.97337 32.671...",6.882354e+07,17090209.8,1059319.37,NaN


In [31]:
# replace nan value with tract level owner income
owners_df['ave_i_0'].fillna((owners_df['owner_i_0_tract']), inplace=True)

/var/folders/2k/vz2b5q6n1m9_qv0lncpx5jww0000gn/T/ipykernel_41937/2598611912.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  owners_df['ave_i_0'].fillna((owners_df['owner_i_0_tract']), inplace=True)


In [32]:
owners_df.drop(columns='owner_i_0_tract').to_csv('data/census_data_incl.csv', index=False)